In [1]:
!pip install transformers datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from google.colab import drive
import torch

# Carregar os dados
drive.mount('/content/drive')
dataset = load_dataset('csv', data_files={
                                          'train': '/content/drive/MyDrive/Colab Notebooks/df_stj_train_fsl_1_shot_PURO.csv',
                                          'test': '/content/drive/MyDrive/Colab Notebooks/df_stj_teste.csv'
                                          })

# Tokenizador e modelo RoBERTa
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7)

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
label_map_train = {'DIREITO ADMINISTRATIVO': 0,
                   'DIREITO CIVIL': 1,
                   'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                   'DIREITO DO CONSUMIDOR': 3,
                   'DIREITO EMPRESARIAL': 4,
                   'DIREITO PENAL': 5,
                   'DIREITO AMBIENTAL': 6
}

label_map_test = {'DIREITO ADMINISTRATIVO': 0,
                  'DIREITO CIVIL': 1,
                  'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                  'DIREITO DO CONSUMIDOR': 3,
                  'DIREITO EMPRESARIAL': 4,
                  'DIREITO PENAL': 5,
                  'DIREITO AMBIENTAL': 6
}

def preprocess_training_data(examples):
    examples["label"] = [label_map_train[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

def preprocess_testing_data(examples):
    examples["label"] = [label_map_test[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

encoded_dataset = {}

encoded_dataset["train"] = dataset["train"].map(preprocess_training_data, batched=True)

# Pré-processando o conjunto de teste
encoded_dataset["test"] = dataset["test"].map(preprocess_testing_data, batched=True)

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

def compute_metrics(pred: EvalPrediction):  # Add type hint for clarity
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)

    # Return a dictionary containing the computed metrics
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/1111 [00:00<?, ? examples/s]

In [4]:
# Configuração do treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"F1 Score: {eval_results['eval_f1']}")
print(f"Precision: {eval_results['eval_precision']}")
print(f"Recall: {eval_results['eval_recall']}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.962059,0.054905,0.005715,0.003015,0.054905
2,No log,1.960970,0.054905,0.005715,0.003015,0.054905
3,No log,1.960673,0.054905,0.005715,0.003015,0.054905


Accuracy: 0.05490549054905491
F1 Score: 0.00571541795817807
Precision: 0.0030146128924323575
Recall: 0.05490549054905491
